# Exploratory Data Analysis

In [ ]:
# Import necessary libraries and functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

from scripts import load_data, analyse_missing_values, drop_unneeded_columns, plot_accidents_by_day_of_week, plot_accidents_by_month, plot_accidents_by_time, plot_distribution_share, preprocess_and_plot_correlation, check_unique_values, drop_missing_values, plot_feature_vs_target


## Load & Preview Dataframe

In [ ]:
# Call function to read dataset
df = load_data()

# Load data head
df.head()

In [ ]:
#Load data shape
df.shape

So basically more than a million accidents, and currently 34 columns here. Let's have a deeper look at the data to investigate how we can proceed.

## Data wrangling

In-depth data overview to look at individual features, observe data type and accordingly process

In [ ]:
#Check information about datatype for individual columns
df.info()

So, quite a good mix of numerical and categorical features here, though a basic overview shows some features have quite a few missing variables. Let's investigate this in-depth.

In [ ]:
# Check for missing values and visualise this
analyse_missing_values(df)


We're seeing quite a few variables with a signficant share of missing variables, we should drop them as it is unlikely they will help in predictive modelling, and any imputes would likely be substantially sensitive to our assumptions, given that these are all objects. These are:
1) Carriageway_Hazards
2) Special_Conditions_at_Site
3) 2nd_Road_Class
4) 1st_Road_Class
5) LSOA_of_Accident_Location

In [ ]:
columns_to_drop = ['Carriageway_Hazards', 'Special_Conditions_at_Site','2nd_Road_Class','1st_Road_Class','LSOA_of_Accident_Location']
df = drop_unneeded_columns(df, columns_to_drop)

We're also seeing quite a few time-related variables, including day of week, time (in hours), year etc. It would be cleaner if we create one DateTime column, so lets do that.

In [ ]:
# Creating a new DateTime variable
df['DateTime'] = df['Date']+' '+ df['Time']

# Convert Datetime to pandas datetime

df['DateTime'] = pd.to_datetime(df['DateTime'], format="%d/%m/%Y %H:%M")

Let's now plot the distribution of the accidents over the day of week, the hours in a day, and the months of the year, to see whether we can discern a trend.

In [ ]:
plot_accidents_by_day_of_week(df, datetime_col='DateTime')
plot_accidents_by_month(df, datetime_col='DateTime')
plot_accidents_by_time(df, datetime_col='DateTime')

Accidents more likely to happen on a Fri, in October & November, and around rush hour (8am, and 4pm to 5pm). So there is a chance that Accident_Severity could be associated with the different time varaibles, simply due to higher frequency at certain points, so we should keep this in. That being said, we have quite a few time variables inside. What we'll do is drop all the time-variables, less DateTime, decompose DateTime into encoded variables for Month, Day of Week, and Hour of Day, for analysis, and then drop DateTime. 

In [ ]:
columns_to_drop2 = ['Date', 'Day_of_Week','Time','Year']
df = drop_unneeded_columns(df, columns_to_drop2)

In [ ]:
df["Month"] = df["DateTime"].dt.month
df["Day_of_Week"] = df["DateTime"].dt.dayofweek  # 0 = Monday, 6 = Sunday
df["Hour_of_Day"] = df["DateTime"].dt.hour

# Drop DateTime
columns_to_drop3 = ['DateTime']
df = drop_unneeded_columns(df, columns_to_drop3)


In [ ]:
df.dtypes

So, we have now our time variables as numeric features within our DataFrame.

## Analysis & Visualisation

Now let's have a look at our target; for this we will look at the values and the distribution within Accident_Severity

In [ ]:
# Check for readings within target variable
unique_target_values = check_unique_values(df, column="Accident_Severity")

In [ ]:
plot_distribution_share(df, 'Accident_Severity')

A large majority of accidents are classified as 'Slight', while roughly 15% of accidents are classidied as 'Serious'. Now, let us check the correlation of our target with the rest of the features, to get a better idea of which features will have better predictive power. We shall do this by creating a correlation matrix. To avoid altering the df that has been modified up till now, the function will create a copy to process, so that we can leave all encoding to the feature engineering aspect of this module.

In [ ]:
# Define the target column and mapping
target_column = "Accident_Severity"
severity_mapping = {"Slight": 0, "Serious": 1, "Fatal": 2}

# Select categorical columns (excluding the target column)
categorical_columns = df.select_dtypes(include=["object"]).columns.tolist()
categorical_columns.remove(target_column)

# Call the function
preprocess_and_plot_correlation(df, target_column, categorical_columns, severity_mapping)

The correlation matrix will be useful in identifying the features we want to use for our modelling, adding some quantitative insights to some intution. For one, it is unlikely that granular location features will have that much of an effect on Accident_Severity, and this is backed by the correlation matrix. As such, we can drop the following indicators, for ease of analysis:

#1. **`2nd_Road_Class`**: secondary road details are less relevant.

#2. **`1st_Road_Class`**: redundant given other road-related features.

#3. **`1st_Road_Number`**: road identifiers unlikely to predict severity.

#4. **`Location_Easting_OSGR`, `Location_Northing_OSGR`, `Latitude`, `Longitude`**: Geographic coordinates too granular.

#5. **`InScotland`**: Binary indicator not relevant given high level overview

#6. **`Police_Force`**: Jurisdiction of police unlikely to affect severity of accident

In [ ]:
# Identifying columns
columns_to_drop4 = ['1st_Road_Number','2nd_Road_Number','Latitude','Longitude',
                      'Local_Authority_(Highway)','Local_Authority_(District)','Location_Easting_OSGR','Location_Northing_OSGR','InScotland', 'Police_Force']


df_filtered = drop_unneeded_columns(df, columns_to_drop4)

In [ ]:
#Previewing data
df_filtered.head()

Let's recreate the correlation matrix again

In [ ]:
# Define the target column and mapping
target_column = "Accident_Severity"
severity_mapping = {"Slight": 0, "Serious": 1, "Fatal": 2}

# Select categorical columns (excluding the target column)
categorical_columns = df_filtered.select_dtypes(include=["object"]).columns.tolist()
categorical_columns.remove(target_column)

# Call the function
preprocess_and_plot_correlation(df_filtered, target_column, categorical_columns, severity_mapping)

And let's have a look at the data types within each column

In [ ]:
df_filtered.dtypes

Now, let's look at the unique values within each column

In [ ]:
unique_values_dict = check_unique_values(df_filtered)

And let's look at the number of missings

In [ ]:
analyse_missing_values(df_filtered)

In [ ]:
df_filtered.shape

Interesting, with a million data points, the largest number of missing's within a column amounts to 21,000. That's relatively small given the number of data points, so rather than going through the trouble of making a category for missing, we can drop them out of the equation entirely, without massively affecting predictive performance.

In [ ]:
df_filtered = drop_missing_values(df_filtered)

In [ ]:
df_filtered.shape

In [ ]:
unique_values_dict = check_unique_values(df_filtered)

That looks much better now, for the Junction_Control, Urban_or_Rural_Area, and Junction_Detail, there technically is a ready made category which denotes missing via the 'Data Missing' or 'Unallocated' fields. This should conclude the processing, and we are ready to now save this as a parquet file to proceed with the rest of the modelling.

In [ ]:
# Save the final processed dataset to a .parquet file
df_filtered.to_parquet("data/processed_data.parquet", index=False)
print("Processed data saved to data/processed_data.parquet")

The below section is additional analysis to look at the distribution of the different features against the target, just to provide some context into how different features could affect the target, along with some multicollinearity instances across different features.

### Investigating the effect of weather conditions

Weather conditions, including rain, has a strong likelihood of increasing accident prevalence, and by extension, severity. 

In [ ]:
# Plot the distribution of the weather variable
plot_distribution_share(df_filtered, "Weather_Conditions")

Interesting, most accidents happened with no adverse weather conditions, let's now look at how the weather conditions vary with our target, to see whether severity could have been affected by weather conditions.

In [ ]:
# Plot the distribution of the feature against the target
plot_feature_vs_target(df_filtered, "Weather_Conditions", "Accident_Severity", kind = "bar")

For each severity, most accidents happened with no adverse weather conditions, though for all other weather conditons less fine and no high winds, this was more evenly distributed, suggesting that there is a chance that weather conditions could have made an impact.

### Investigating the effect of road conditions

Perhaps weather conditions is more of a secondary variable influencing road conditions (as evidenced by the relatively stronger correlation between the two variables in our correlation matrix). Wet roads due to heavy rain, or icy roads due to snow could have significant affect on a drive.

In [ ]:
# Plot the distribution of the road surface variable
plot_distribution_share(df_filtered, "Road_Surface_Conditions")

Most accidents happened on dry roads, but we're also seeing quite a few accidents on wet or damp roads, which aligns with our hypthesis

In [ ]:
# Plot the distribution of the road surface conditions against weather conditions
plot_feature_vs_target(df_filtered, "Road_Surface_Conditions", "Weather_Conditions", kind = "bar")

Again, our hypothesis seems right here, wet weather conditions lead to wet roads, which affect drive

In [ ]:
# Plot the distribution of the road surface conditions against weather conditions
plot_feature_vs_target(df_filtered, "Road_Surface_Conditions", "Accident_Severity", kind = "bar")

### Looking into rural-urban splits

In [ ]:
# Calculate the proportion of each severity within Urban/Rural areas
proportions = (
    df_filtered.groupby("Urban_or_Rural_Area")["Accident_Severity"]
    .value_counts(normalize=True)
    .rename("Proportion")
    .reset_index()
)

# Plot the proportions
sns.barplot(
    x="Urban_or_Rural_Area", 
    y="Proportion", 
    hue="Accident_Severity", 
    data=proportions
)
plt.title("Share of Accident Severity by Urban vs Rural Area")
plt.ylabel("Proportion")
plt.xlabel("Urban or Rural Area")
plt.legend(title="Accident Severity", loc="upper right")
plt.show()

Rural areas more likely to be associated with serious and fatal accidents, likely due to to higher speed limits into rural areas, which explains the high correlation between the two variables.

In [ ]:
sns.boxplot(x="Accident_Severity", y="Speed_limit", data=df_filtered)
plt.title("Speed Limit vs Accident Severity")
plt.show()

Seems like accident severity rating increases with speed limit, at least from slight, to severe and fatal. Higher speed thresholds associated with impact, which would in turn feed into severity

In [ ]:
sns.boxplot(x="Accident_Severity", y="Number_of_Casualties", data=df_filtered)
plt.title("Number of Casualties vs Accident Severity")
plt.show()

Casualities dosen't have an explicit link with severtiy, though 'Fatal' accidents sees greater outlier magnittude, and on average, Severe and Fatal accidents have wider boxes, indicating larger spread.

In [ ]:
# Group by Accident_Severity and Police_Attendance to calculate proportions
attendance_severity = df_filtered.groupby(["Accident_Severity", "Did_Police_Officer_Attend_Scene_of_Accident"]).size().unstack()

# Normalize to calculate the percentage share of each attendance category
attendance_share = attendance_severity.div(attendance_severity.sum(axis=1), axis=0)

# Plot the stacked bar chart
attendance_share.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="viridis")
plt.title("Share of Police Attendance Categories by Accident Severity")
plt.xlabel("Accident Severity")
plt.ylabel("Share")
plt.legend(title="Police Attendance", loc="upper right")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

For context,

1.0 = police attended;
2.0 = police did not attend
3.0 = processed via self-completion form

It seems like police attendance was more likely for increasing scales of accident severity